In [1]:
import json, hashlib, base64
from mysql.connector import pooling

with open('../../mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
pool = pooling.MySQLConnectionPool(pool_name="mypool", pool_size=3, **config)

In [2]:
def get_board(bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "SELECT b.bid, b.uid, b.title, b.content, b.modTime, b.viewCount, b.replyCount, b.files, u.uname \
        FROM board AS b JOIN users AS u ON b.uid=u.uid WHERE b.bid=%s"
    cur.execute(sql,(bid,))
    row = cur.fetchone()
    cur.close()
    conn.close()
    return row

In [3]:
b = get_board(1)
b

(1,
 'james',
 'test title',
 'contents',
 datetime.datetime(2023, 8, 16, 18, 13, 56),
 0,
 0,
 '{"list":[]}',
 '제임스')

In [3]:
field = 'title'
query = ''

In [4]:
def get_board_count(field, query):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "select count(bid) from board where isDeleted=0 AND %s like %s"
    cur.execute(sql,(field,f'%{query}%'))
    row = cur.fetchone()
    cur.close()
    conn.close()
    return row

In [9]:
a = get_board_count(field=field, query=query)[0]
a

12

In [6]:
def insert_board(params):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "insert into board values(default, %s, %s, %s, default, default, default, default, %s)"
    cur.execute(sql,params)
    row = cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return row

In [7]:
params = ('james','untitle','contents','{"list":[]}')

In [8]:
c = insert_board(params)

In [10]:
def update_board(params):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "update board set title=%s, content=%s, modTime=now(), files=%s where bid=%s"
    cur.execute(sql,params)
    row = cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return row

In [13]:
params = ('title','contents','{"list":[]}',9)

In [14]:
update_board(params)

In [15]:
def delete_board(bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "UPDATE board SET isDeleted=1 WHERE bid=%s"
    cur.execute(sql,(bid,))
    row = cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return row

In [16]:
delete_board(1)

In [6]:
def increase_count(field,bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = f"UPDATE board SET {field}={field}+1 where bid = %s"
    cur.execute(sql,(bid,))
    cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()


In [7]:
increase_count(field='viewCount',bid=2)

In [8]:
def decrease_reply_count(bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "UPDATE board SET replyCount=replyCOunt-1 where bid = %s"
    cur.execute(sql,(bid,))
    cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()

In [9]:
increase_count(field='replyCount',bid=2)

In [10]:
decrease_reply_count(2)

In [2]:
def get_reply_list(bid):
    
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "SELECT r.rid, r.`comment`, r.regTime, r.isMine, r.uid, r.bid, u.uname \
        FROM reply AS r JOIN users AS u ON r.uid=u.uid WHERE bid=%s"
    cur.execute(sql,(bid,))
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return rows

In [5]:
cc = get_reply_list(17)

In [6]:
for c in cc:
    print(c)

(1, '노잼', datetime.datetime(2023, 8, 18, 19, 5, 2), 1, 'james', 17, '제임스')
(2, '삭제해주세요', datetime.datetime(2023, 8, 18, 19, 5, 41), 0, 'sara', 17, '사라')


In [10]:
def insert_reply(params):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "INSERT INTO reply VALUES (default, %s, default, %s, %s, %s)"
    cur.execute(sql,params)
    row = cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return row

In [11]:
params = ('test_re',0,'lucy',17)

In [12]:
insert_reply(params)

In [3]:
def get_board_list(field, query, offset):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "SELECT b.bid, b.uid, b.title, b.modTime, b.viewCount, b.replyCount, u.uname FROM board AS b \
        JOIN users AS u ON b.uid=u.uid WHERE b.isDeleted=0 AND %s LIKE %s ORDER BY b.modTime DESC LIMIT 10 OFFSET %s"
    cur.execute(sql,(field,f'%{query}%', offset))
    row = cur.fetchall()
    cur.close()
    conn.close()
    return row

In [4]:
qq = get_board_list('title','',1)

In [5]:
for q in qq:
    print(q)

(17, 'james', 'untitle', datetime.datetime(2023, 8, 18, 0, 14, 14), 0, 0, '제임스')
(12, 'sadie', 'shotgun', datetime.datetime(2023, 8, 17, 23, 13, 53), 0, 0, '세이디')
(11, 'spring', 'summer', datetime.datetime(2023, 8, 17, 23, 12, 56), 0, 0, '스프링')
(10, 'segue', 'fan service', datetime.datetime(2023, 8, 17, 23, 12, 18), 0, 0, '세구')
(8, 'whale', '고래', datetime.datetime(2023, 8, 17, 23, 10, 38), 0, 0, '웨일')
(7, 'arthur', '블랙워터에', datetime.datetime(2023, 8, 17, 23, 10, 8), 0, 0, '아서')
(6, 'spring', '사실 오늘은', datetime.datetime(2023, 8, 17, 23, 9, 20), 0, 0, '스프링')
(4, 'kate', 'k-title', datetime.datetime(2023, 8, 17, 23, 8, 38), 0, 0, '케이트')
(3, 'adler', 'devil run', datetime.datetime(2023, 8, 16, 18, 16, 29), 0, 0, '애들러')
(2, 'lucy', 'lucy test', datetime.datetime(2023, 8, 16, 18, 14, 28), 2, 0, '루시')


In [26]:
qq[0][3]

datetime.datetime(2023, 8, 18, 0, 14, 14)

In [23]:
from datetime import date
rrrrrr = date.today().isoformat()
rrrrrr

'2023-08-18'

In [45]:
from datetime import datetime
today = datetime.today()
today


datetime.datetime(2023, 8, 18, 22, 58, 36, 995557)

In [35]:
today = '2023-08-17 23:12:56'

In [36]:
if today == qq[0][3]:
    print('b')
else:
    print('a')

a


In [49]:
today = datetime.today(qq[0][3])

TypeError: datetime.today() takes no arguments (1 given)